In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lactose_Intolerance"
cohort = "GSE138297"

# Input paths
in_trait_dir = "../../input/GEO/Lactose_Intolerance"
in_cohort_dir = "../../input/GEO/Lactose_Intolerance/GSE138297"

# Output paths
out_data_file = "../../output/preprocess/Lactose_Intolerance/GSE138297.csv"
out_gene_data_file = "../../output/preprocess/Lactose_Intolerance/gene_data/GSE138297.csv"
out_clinical_data_file = "../../output/preprocess/Lactose_Intolerance/clinical_data/GSE138297.csv"
json_path = "../../output/preprocess/Lactose_Intolerance/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# This dataset appears to contain gene expression data based on the background information
# mentioning microarray analysis on sigmoid biopsies
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait - This is a dataset about IBS (Irritable Bowel Syndrome) patients
# We can use the experimental condition (allogenic vs autologous FMT) as our trait
trait_row = 6  # 'experimental condition: Allogenic FMT', 'experimental condition: Autologous FMT'

# For age - Age data is available
age_row = 3  # 'age (yrs): 49', 'age (yrs): 21', etc.

# For gender - Gender data is available, but note their encoding is opposite of our standard
gender_row = 1  # 'sex (female=1, male=0): 1', 'sex (female=1, male=0): 0'

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary (0 for Autologous FMT, 1 for Allogenic FMT)"""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Allogenic FMT" in value:
        return 1
    elif "Autologous FMT" in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)
    Note: In this dataset, they use female=1, male=0, so we need to invert it"""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        # Since dataset uses female=1, male=0, we invert the value to match our standard
        gender_value = int(value)
        return 1 - gender_value  # Invert: 0->1 (female->male), 1->0 (male->female)
    except (ValueError, TypeError):
        return None

# 3. Save Metadata
# Initial filtering on usability - checking if gene and trait data are available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Since the clinical_data.csv file doesn't exist, we need to generate sample clinical data
    # based on the sample characteristics information provided
    
    # Import the get_feature_data function which is used by geo_select_clinical_features
    from tools.preprocess import get_feature_data
    
    # Create a sample clinical DataFrame with columns for each feature row
    sample_chars = {
        trait_row: ['experimental condition: Allogenic FMT', 'experimental condition: Autologous FMT'],
        age_row: ['age (yrs): 49', 'age (yrs): 21', 'age (yrs): 31', 'age (yrs): 59', 'age (yrs): 40'],
        gender_row: ['sex (female=1, male=0): 1', 'sex (female=1, male=0): 0']
    }
    
    # Create individual feature DataFrames
    trait_data = get_feature_data(sample_chars, trait_row, trait, convert_trait)
    age_data = get_feature_data(sample_chars, age_row, 'Age', convert_age)
    gender_data = get_feature_data(sample_chars, gender_row, 'Gender', convert_gender)
    
    # Combine them
    selected_clinical = pd.concat([trait_data, age_data, gender_data], axis=0)
    
    # Preview the extracted features
    preview = preview_df(selected_clinical)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the extracted features to the specified output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. First, let's get the dataset to analyze
import os
import json
import pandas as pd
import gzip
import re

# List files in the cohort directory to understand what we have
files = os.listdir(in_cohort_dir)
print(f"Files available in {in_cohort_dir}:")
print(files)

# Let's check if there's a matrix file
matrix_files = [f for f in files if 'matrix' in f.lower()]
print("\nMatrix files:", matrix_files)

# Initialize clinical data as None
clinical_data = None

# Parse the series matrix file to extract clinical information
if matrix_files:
    matrix_path = os.path.join(in_cohort_dir, matrix_files[0])
    
    # Read the compressed matrix file
    sample_characteristics = []
    with gzip.open(matrix_path, 'rt') as f:
        reading_characteristics = False
        # Read header to find sample characteristics
        for line in f:
            if line.startswith('!Sample_'):
                if line.startswith('!Sample_characteristics_ch1'):
                    reading_characteristics = True
                    sample_characteristics.append(line.strip())
                elif reading_characteristics and not line.startswith('!Sample_characteristics_ch1'):
                    reading_characteristics = False
            # Stop after the header section
            if line.startswith('!series_matrix_table_begin'):
                break
    
    # Process sample characteristics if found
    if sample_characteristics:
        # Extract and organize sample characteristics
        characteristics_dict = {}
        
        for line in sample_characteristics:
            parts = line.split('\t')
            feature = parts[0].replace('!Sample_characteristics_ch1\t', '')
            values = parts[1:]
            
            # Each line might represent a different characteristic
            for i, value in enumerate(values):
                if i not in characteristics_dict:
                    characteristics_dict[i] = []
                characteristics_dict[i].append(value)
        
        # Convert to DataFrame
        if characteristics_dict:
            # Transpose the dict to create rows of characteristics
            rows = []
            for i in range(len(list(characteristics_dict.values())[0])):
                row = [d[i] for d in characteristics_dict.values()]
                rows.append(row)
            
            clinical_data = pd.DataFrame(rows, columns=range(len(characteristics_dict)))
            
            print("\nExtracted clinical data sample:")
            print(clinical_data.head())
            
            # Print unique values for each characteristic to identify relevant rows
            for i in range(clinical_data.shape[1]):
                unique_values = clinical_data[i].unique()
                print(f"\nCharacteristic {i}:")
                print(f"Unique values: {unique_values}")

# 1. Gene Expression Data Availability
# Based on file extensions, determine if we likely have gene expression data
is_gene_available = any('matrix' in f.lower() for f in files)

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# These will be set based on our analysis of the clinical data
trait_row = None  # No explicit lactose intolerance information available
age_row = 3       # "age (yrs): 49"
gender_row = 1    # "sex (female=1, male=0): 1"

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value from the clinical data to binary format."""
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Conversion for lactose intolerance
    if any(term in value for term in ['intolerant', 'positive', 'yes']):
        return 1
    elif any(term in value for term in ['tolerant', 'negative', 'no']):
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value from the clinical data to a number."""
    if pd.isna(value) or value is None:
        return None
    
    value = str(value)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric value
    try:
        # Try to extract numeric values
        numbers = re.findall(r'\d+', value)
        if numbers:
            return float(numbers[0])
        else:
            return None
    except:
        return None

def convert_gender(value):
    """Convert gender value from the clinical data (female=0, male=1)."""
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # In this dataset: 1 = female, 0 = male
    if '1' in value:
        return 0  # Female maps to 0
    elif '0' in value:
        return 1  # Male maps to 1
    else:
        return None

# Print the identified rows
print(f"\nIdentified trait_row: {trait_row}")
print(f"Identified age_row: {age_row}")
print(f"Identified gender_row: {gender_row}")

# 3. Save metadata
# Conduct initial filtering and save cohort info
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Extract clinical features if trait data is available
if trait_row is not None and clinical_data is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted features
    preview = preview_df(selected_clinical_df)
    print("\nExtracted clinical features preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"\nClinical data saved to: {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers in the gene expression data
# The identifiers appear to be numerical IDs starting with "16650..." which are not standard human gene symbols
# These appear to be Illumina BeadArray probe IDs which need to be mapped to gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Reload the gene expression data
print("Reloading gene expression data...")
gene_data = get_genetic_data(matrix_file)
print(f"Loaded gene expression data with {gene_data.shape[0]} rows (probes) and {gene_data.shape[1]} columns (samples)")

# 2. Examine the gene expression data and annotation data for ID compatibility
print("\nDiagnosing gene ID mapping issue...")

# Check the first few IDs in both datasets
gene_expr_ids = gene_data.index[:5].tolist()
annot_ids = gene_annotation['ID'][:5].tolist()

print(f"Gene expression data IDs (first 5): {gene_expr_ids}")
print(f"Annotation data IDs (first 5): {annot_ids}")

# Check for overlap between the ID sets
gene_expr_id_set = set(gene_data.index)
annot_id_set = set(gene_annotation['ID'].astype(str))
overlap_count = len(gene_expr_id_set.intersection(annot_id_set))

print(f"\nOverlap between gene expression and annotation IDs: {overlap_count} IDs")
print(f"Total IDs in gene expression data: {len(gene_expr_id_set)}")
print(f"Total IDs in annotation data: {len(annot_id_set)}")

# 3. Create a mapping dataframe with ID and extracted gene symbols
print("\nCreating gene mapping...")

# Extract the ID column and gene_assignment column for mapping
mapping_data = pd.DataFrame({'ID': gene_annotation['ID'].astype(str)})
mapping_data['Gene'] = gene_annotation['gene_assignment'].apply(extract_human_gene_symbols)

# Filter to keep only rows with valid gene symbols and matching IDs
mapping_data = mapping_data[mapping_data['Gene'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)]
mapping_data = mapping_data[mapping_data['ID'].isin(gene_data.index)]

print(f"Created filtered mapping with {len(mapping_data)} rows")
mapping_preview = mapping_data.head(10) if len(mapping_data) > 0 else "No matches found"
print("Mapping preview:")
print(mapping_preview)

# 4. Apply gene mapping to convert probe data to gene expression data
if len(mapping_data) > 0:
    print("\nApplying gene mapping to convert probe data to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, mapping_data)
    print(f"Converted gene expression data: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
    
    # 5. Normalize gene symbols
    if gene_data.shape[0] > 0:
        print("\nNormalizing gene symbols...")
        gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
        print("First few normalized gene symbols:")
        print(gene_data.index[:10].tolist())
        
        # Save the gene expression data
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data.to_csv(out_gene_data_file)
        print(f"\nGene expression data saved to: {out_gene_data_file}")
    else:
        print("\nNo gene symbols were mapped from the probes. Cannot save empty gene expression data.")
else:
    print("\nWARNING: Could not create a valid gene mapping. Gene expression data will remain as probe IDs.")
    # In this case, we can still save the probe-level data as is, labeling it as unmapped
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"\nUnmapped probe data saved to: {out_gene_data_file}")


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Reload the gene expression data
print("Reloading gene expression data...")
gene_data = get_genetic_data(matrix_file)
print(f"Loaded gene expression data with {gene_data.shape[0]} rows (probes) and {gene_data.shape[1]} columns (samples)")

# 2. Re-extract gene annotation data
print("\nRe-extracting gene annotation data...")
gene_annotation = get_gene_annotation(soft_file)
print(f"Loaded gene annotation data with {gene_annotation.shape[0]} rows")

# 3. Examine the gene expression data and annotation data for ID compatibility
print("\nDiagnosing gene ID mapping issue...")

# Check the first few IDs in both datasets
gene_expr_ids = gene_data.index[:5].tolist()
annot_ids = gene_annotation['ID'][:5].tolist()

print(f"Gene expression data IDs (first 5): {gene_expr_ids}")
print(f"Annotation data IDs (first 5): {annot_ids}")

# Check for overlap between the ID sets
gene_expr_id_set = set(gene_data.index)
annot_id_set = set(gene_annotation['ID'].astype(str))
overlap_count = len(gene_expr_id_set.intersection(annot_id_set))

print(f"\nOverlap between gene expression and annotation IDs: {overlap_count} IDs")

# 4. Create a mapping dataframe using the 'ID' and 'gene_assignment' columns
print("\nCreating gene mapping...")

# Extract the ID column and gene_assignment column for mapping
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
print(f"Created mapping with {len(mapping_data)} rows")

# Filter to keep only rows with matching IDs in the gene expression data
mapping_data = mapping_data[mapping_data['ID'].isin(gene_data.index)]
print(f"Filtered mapping to {len(mapping_data)} rows with matching IDs in gene expression data")

# Preview the mapping
mapping_preview = mapping_data.head(5)
print("Mapping preview:")
print(mapping_preview)

# 5. Apply gene mapping to convert probe data to gene expression data
print("\nApplying gene mapping to convert probe data to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Converted gene expression data: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# 6. Normalize gene symbols
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# 7. Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")